#import library

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_date
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, dayofyear, weekofyear
from pyspark.sql.window import Window

#Open the file

In [0]:
filepath = 's3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json'

steam_df = spark.read.format('json').load(filepath, multiline=True)



# first analysis

In [0]:
steam_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

In [0]:
steam_df.show(5)

+--------------------+-------+
|                data|     id|
+--------------------+-------+
|{10, [Multi-playe...|     10|
|{1000000, [Single...|1000000|
|{1000010, [Single...|1000010|
|{1000030, [Multi-...|1000030|
|{1000040, [Single...|1000040|
+--------------------+-------+
only showing top 5 rows



In [0]:
steam_describe = steam_df.describe()
steam_describe.toPandas()

,summary,id
0,count,55691
1,mean,1025603.0926720655
2,stddev,522784.96832834504
3,min,10
4,max,999990


In [0]:
#correct ??

def missing_value(col):
 return F.sum(F.col(col).isNull().cast('int')).alias(col)

missing_values = steam_df.select([missing_value(col) for col in steam_df.columns]).toPandas()
missing_values

,data,id
0,0,0


# Analysis at the "macro" level

#### Which publisher has released the most games on Steam?

In [0]:
best_publisher = steam_df\
    .groupBy(F.col('data.publisher'))\
    .count()\
    .orderBy(F.desc('count'))

best_publisher.show(5)

+--------------+-----+
|     publisher|count|
+--------------+-----+
|Big Fish Games|  422|
|        8floor|  202|
|          SEGA|  165|
|Strategy First|  151|
|   Square Enix|  141|
+--------------+-----+
only showing top 5 rows



#### What are the best rated games?

In [0]:
best_rated_games = steam_df\
    .select(F.col('data.name'),F.col('data.positive'))\
    .orderBy(F.desc('data.positive'))
best_rated_games.show(5)

+--------------------+--------+
|                name|positive|
+--------------------+--------+
|Counter-Strike: G...| 5943345|
|              Dota 2| 1534895|
|  Grand Theft Auto V| 1229265|
| PUBG: BATTLEGROUNDS| 1185361|
|            Terraria| 1014711|
+--------------------+--------+
only showing top 5 rows



#### Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

In [0]:
#first reading of the release_date :  2022/11/7
steam_df\
    .select(F.col('data.release_date'))\
    .orderBy(F.desc('release_date'))\
    .show(5)

+------------+
|release_date|
+------------+
|   2022/11/7|
|   2022/11/7|
|   2022/11/7|
|   2022/11/6|
|   2022/11/6|
+------------+
only showing top 5 rows



In [0]:
#need to change the release_date format to use it

steam_df_time = steam_df.select(F.col('data.release_date').alias('release_date')) \
    .withColumn('release_date', F.to_date(F.col('release_date'), 'y/M/d')) \
    .withColumn('release_year', year(F.col('release_date')))

#to check if it's OK
steam_df_time.printSchema()
steam_df_time.select('release_year').orderBy(F.desc('release_year')).show(5)

root
 |-- release_date: date (nullable = true)
 |-- release_year: integer (nullable = true)

+------------+
|release_year|
+------------+
|        2022|
|        2022|
|        2022|
|        2022|
|        2022|
+------------+
only showing top 5 rows



In [0]:
releases_by_year = steam_df_time\
    .groupBy('release_year')\
    .count()\
    .orderBy(F.desc('release_year'))

releases_by_year.show(10)

+------------+-----+
|release_year|count|
+------------+-----+
|        2022| 7451|
|        2021| 8805|
|        2020| 8287|
|        2019| 6949|
|        2018| 7663|
|        2017| 6006|
|        2016| 4176|
|        2015| 2566|
|        2014| 1550|
|        2013|  469|
+------------+-----+
only showing top 10 rows



#### How are the prizes distributed? Are there many games with a discount?

In [0]:
price_discount_df = steam_df \
    .select((F.col('data.initialprice').cast('float').alias('initialprice')),(F.col('data.price').cast('float').alias('price')),(F.col('data.discount').cast('integer').alias('discount')))  \
    .withColumn('initialprice_$', F.col('initialprice') / 100) \
    .drop('initialprice') \
    .withColumn('price_$', F.col('price') / 100) \
    .drop('price') \

price_discount_df.show(5)

display(price_discount_df.describe().toPandas())

games_with_discount = price_discount_df.filter(F.col('discount') > 0).count()
print(f"number of games with discount: {games_with_discount}")

expensive_game_or_mistackes = price_discount_df.filter(F.col('price_$') > 100).count()
print(f"number of games more with price more than 100 $: {expensive_game_or_mistackes}")

+--------+--------------+-------+
|discount|initialprice_$|price_$|
+--------+--------------+-------+
|       0|          9.99|   9.99|
|       0|          9.99|   9.99|
|      70|         19.99|   5.99|
|       0|         19.99|  19.99|
|       0|          1.99|   1.99|
+--------+--------------+-------+
only showing top 5 rows



summary,discount,initialprice_$,price_$
count,55691,55691,55691
mean,2.603777989262179,7.975663033519214,7.732849832104521
stddev,12.887080174743176,11.047624778413402,10.931345827234507
min,0,0.0,0.0
max,90,999.0,999.0


number of games with discount: 2518
number of games more with price more than 100 $: 37


#### What are the most represented languages?

In [0]:
languages_df = steam_df\
    .select(F.col('data.languages')).alias('languages')\
    .withColumn("list_languages",F.split(('languages'),","))\
    .withColumn('exp_languages', F.explode('list_languages'))\
    .groupBy('exp_languages')\
    .count()\
    .orderBy(F.desc('count'))

languages_df.show(10)

+--------------------+-----+
|       exp_languages|count|
+--------------------+-----+
|             English|54646|
|              German|13996|
|              French|13406|
|             Russian|12839|
|     Spanish - Spain|12224|
|  Simplified Chinese|12213|
|            Japanese|10170|
|             Italian| 9297|
| Portuguese - Brazil| 6739|
|              Korean| 6575|
+--------------------+-----+
only showing top 10 rows



#### Are there many games prohibited for children under 16/18?

In [0]:
ages_df = steam_df\
    .select(F.col('data.required_age').cast('integer').alias('required_age'))\
    .filter((F.col('required_age') == 16) | (F.col('required_age') == 18))\
    .groupBy('required_age')\
    .count()\
    .orderBy(F.desc('count'))

ages_df.show(10)

+------------+-----+
|required_age|count|
+------------+-----+
|          18|  223|
|          16|   38|
+------------+-----+



# Genres analysis

In [0]:
#dataframe for genre

genre_df = steam_df\
    .select((F.col('data.positive').cast('integer').alias('positive')),
            (F.col('data.negative').cast('integer').alias('negative')),
            (F.col('data.genre')).alias('genre'),
            (F.col('data.publisher')).alias('publisher'),
            (F.col('data.initialprice').cast('float').alias('initialprice')),
            (F.col("data.platforms.linux")),
            (F.col("data.platforms.mac")),
            (F.col("data.platforms.windows")))\
    .withColumn("list_genre",F.split(('genre'),","))\
    .withColumn('exp_genre', F.explode('list_genre'))\
    .withColumn("genre_clean",F.lower(F.regexp_replace('exp_genre', "\\s+", "")))\
    .withColumn('initialprice_$', F.col('initialprice') / 100) \
    .drop('genre', 'list_genre','exp_genre','initialprice')\
    .withColumnRenamed('genre_clean', 'genre')


genre_df.show()

+--------+--------+--------------------+-----+-----+-------+----------+--------------+
|positive|negative|           publisher|linux|  mac|windows|     genre|initialprice_$|
+--------+--------+--------------------+-----+-----+-------+----------+--------------+
|  201215|    5199|               Valve| true| true|   true|    action|          9.99|
|      27|       5|PsychoFlux Entert...|false|false|   true|    action|          9.99|
|      27|       5|PsychoFlux Entert...|false|false|   true| adventure|          9.99|
|      27|       5|PsychoFlux Entert...|false|false|   true|     indie|          9.99|
|    4032|     646|Team17, NEXT Studios|false|false|   true| adventure|         19.99|
|    4032|     646|Team17, NEXT Studios|false|false|   true|     indie|         19.99|
|    4032|     646|Team17, NEXT Studios|false|false|   true|       rpg|         19.99|
|    4032|     646|Team17, NEXT Studios|false|false|   true|  strategy|         19.99|
|    1575|     115| Vertigo Gaming Inc.|fal

#### What are the most represented genres?

In [0]:
most_represented_genre = genre_df\
    .select(F.col('genre'))\
    .groupBy('genre')\
    .count()\
    .orderBy(F.desc('count'))

most_represented_genre.show(10)

+-----------+-----+
|      genre|count|
+-----------+-----+
|      indie|39681|
|     action|23759|
|     casual|22086|
|  adventure|21431|
|   strategy|10895|
| simulation|10836|
|        rpg| 9534|
|earlyaccess| 6145|
| freetoplay| 3393|
|     sports| 2666|
+-----------+-----+
only showing top 10 rows



#### Are there any genres that have a better positive/negative review ratio?

In [0]:
positive_negative_ratio = genre_df\
    .select(
        (F.col('positive')),
        (F.col('negative')),
        (F.col('genre')))\
    .groupBy("genre")\
    .agg(
        F.sum('positive').alias('total_positive'),
        F.sum('negative').alias('total_negative'))\
    .withColumn('positive_nevative_ratio', F.round(F.col('total_positive')/ F.col('total_negative'),1))\
    .orderBy(F.desc('positive_nevative_ratio'))

positive_negative_ratio.show()

+-------------------+--------------+--------------+-----------------------+
|              genre|total_positive|total_negative|positive_nevative_ratio|
+-------------------+--------------+--------------+-----------------------+
|       photoediting|        577751|         13745|                   42.0|
| animation&modeling|        690765|         26392|                   26.2|
|design&illustration|        674057|         27007|                   25.0|
|          utilities|        739335|         43503|                   17.0|
|    gamedevelopment|         27461|          3274|                    8.4|
|              indie|      32531023|       4241234|                    7.7|
|    audioproduction|         69118|          9428|                    7.3|
|    videoproduction|        111514|         16362|                    6.8|
|                   |        189649|         27744|                    6.8|
|             casual|      10034967|       1537296|                    6.5|
|      webpu

In [0]:
most_rating = genre_df\
    .select('positive',
            "genre")\
    .groupBy("genre")\
    .agg(F.sum('positive').alias('total_positive_by_genre'))\
    .orderBy(F.desc('total_positive_by_genre'))\
    .show()

+--------------------+-----------------------+
|               genre|total_positive_by_genre|
+--------------------+-----------------------+
|              action|               54858618|
|               indie|               32531023|
|           adventure|               29689445|
|                 rpg|               19425528|
|          freetoplay|               18722246|
|          simulation|               15572390|
|            strategy|               13402870|
|              casual|               10034967|
|massivelymultiplayer|                7979078|
|         earlyaccess|                4334595|
|              sports|                2949805|
|              racing|                2340353|
|           utilities|                 739335|
|  animation&modeling|                 690765|
| design&illustration|                 674057|
|        photoediting|                 577751|
|                    |                 189649|
|     videoproduction|                 111514|
|     audiopr

#### Do some publishers have favorite genres?

In [0]:
# window for ranking
window_spec = Window\
    .partitionBy("publisher")\
    .orderBy(F.desc("genre_count"))

#create new df with count and using windows
publisher_favorite_genre = genre_df\
    .select(
        F.col('publisher'),
        F.col('genre'))\
    .withColumn("genre_count", F.count("genre").over(Window.partitionBy("publisher", "genre")))\
    .withColumn("rank", F.row_number().over(window_spec))\
    .filter(F.col("rank") == 1)\
    .select("publisher", "genre")\
    .distinct()

publisher_favorite_genre.show()

+--------------------+-----------+
|           publisher|      genre|
+--------------------+-----------+
|           AK Studio|      indie|
|         Alon Zubina|  adventure|
|        Appnori Inc.|earlyaccess|
| Art Games Studio...|     casual|
|      Asterion Games|      indie|
|           BBB Games|     action|
|   Beach Day Studios|     action|
|               CGevo|     action|
| Dipl. Ing. Rolf-...|  adventure|
|        FurGoldGames|  adventure|
|     Game for people|     action|
|  Genuine Studio Ltd|  adventure|
| H2 Interactive C...|     action|
| Hangover Cat Pur...|      indie|
|             Iful GS|      indie|
| Indevelopment Pr...|     casual|
|            MV Games|      indie|
|           Metal Fox|      indie|
| Paper Bunker s.r.o.|     action|
|      PeriScope Game|      indie|
+--------------------+-----------+
only showing top 20 rows



#### What are the most lucrative genres?

In [0]:
most_lucrative_genre = genre_df\
    .select('genre',
            "initialprice_$")\
    .groupBy("genre")\
    .agg(F.round(F.sum('initialprice_$'),2).alias('total_$_by_genre'))\
    .orderBy(F.desc('total_$_by_genre'))\
    .show()
    

+--------------------+----------------+
|               genre|total_$_by_genre|
+--------------------+----------------+
|               indie|        269797.1|
|              action|       189541.45|
|           adventure|        178020.6|
|              casual|       128163.07|
|          simulation|       101645.99|
|            strategy|        94404.21|
|                 rpg|        89283.94|
|         earlyaccess|        54529.18|
|              sports|        24682.79|
|              racing|        18260.89|
|           utilities|         7987.41|
| design&illustration|          7933.9|
|massivelymultiplayer|         7598.45|
|  animation&modeling|         6153.87|
|     videoproduction|         4893.52|
|           education|         4624.32|
|     audioproduction|         4051.98|
|     gamedevelopment|         3396.12|
|    softwaretraining|         3238.93|
|        photoediting|         2144.05|
+--------------------+----------------+
only showing top 20 rows



# Platform analysis

In [0]:
plateform_df = steam_df\
    .select(
        (F.col("data.platforms.linux")),
        (F.col("data.platforms.mac")),
        (F.col("data.platforms.windows")))


#### Are most games available on Windows/Mac/Linux instead?

In [0]:
#for only 3 platforms, no need to do more complicated

games_available_platform = plateform_df\
    .agg(
        F.sum(F.col('linux').cast('integer')).alias('games_on_linux'),
        F.sum(F.col('mac').cast('integer')).alias('games_on_mac'),
        F.sum(F.col('windows').cast('integer')).alias('games_on_windows')
    )
games_available_platform.show()

+--------------+------------+----------------+
|games_on_linux|games_on_mac|games_on_windows|
+--------------+------------+----------------+
|          8458|       12770|           55676|
+--------------+------------+----------------+



#### Do certain genres tend to be preferentially available on certain platforms?

In [0]:
# Définir la fenêtre pour trier les genres par count dans chaque plateforme
window_spec = Window.partitionBy("platform").orderBy(F.col("count").desc())

genre_preferentially_platform = genre_df\
    .selectExpr("genre", 
                "stack(3, 'linux', linux, 'mac', mac, 'windows', windows) as (platform, available)")\
    .filter("available == true")\
    .groupBy("platform", "genre")\
    .count()\
    .withColumn("rank", F.row_number().over(window_spec))\
    .filter(F.col("rank") <= 5)\
    .groupBy("platform") \
    .agg(F.collect_list("genre").alias("Top_5_genres"))


genre_preferentially_platform.show(truncate=False)

+--------+--------------------------------------------+
|platform|Top_5_genres                                |
+--------+--------------------------------------------+
|linux   |[indie, action, casual, adventure, strategy]|
|mac     |[indie, casual, adventure, action, strategy]|
|windows |[indie, action, casual, adventure, strategy]|
+--------+--------------------------------------------+

